## Widgets And Dashboards
**Goal:** This notebook will demonstrate how we can use code output from Apache Toree and integrate into some front end application.

We are going to use the [Jupyter Declarative Widgets](https://github.com/jupyter-incubator/declarativewidgets) and [Jupyter Dashboards](https://github.com/jupyter-incubator/dashboards) extensions to build a dynamic dashboard to display our data.  This dashboard will display the top 10 hacker news stories. The user will then be able to "drill down" by clicking on the story. This will then show the comments, word counts, and links from the comments.

This demonstration does not go into detail about deploying dashboards. However, if users are interested in how acheive such results please refer to the [Jupyter Dashboards Server](https://github.com/jupyter-incubator/dashboards_server) and the [Jupyter Dashboard Bundlers](https://github.com/jupyter-incubator/dashboards_bundlers) extensions.

Most of the code stays the same. We will just need to add markup for the display and some wrapper functions to get the data in our widgets.

In [ ]:
%adddeps org.jsoup jsoup 1.9.2 --transitive
%adddeps com.github.seratch hackernews4s_2.10 0.6.0 --transitive
%addjar http://localhost:8888/nbextensions/declarativewidgets/declarativewidgets.jar

In [ ]:
val sqlC = sqlContext

In [ ]:
import hackernews4s.v0._
import sqlC.implicits._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.Row
import org.jsoup.Jsoup
import org.jsoup.nodes.Document
import org.jsoup.nodes.Element
import scala.collection.JavaConversions._
import org.apache.spark.ml.feature.{HashingTF, IDF, Tokenizer}
import org.apache.spark.ml.feature.StopWordsRemover
import org.apache.spark.ml.feature.{CountVectorizer, CountVectorizerModel}
import org.apache.spark.sql.DataFrame
import org.apache.spark.rdd.RDD

In [ ]:
case class Comment(story: Long, id: Long, text: String)

val tokenizer = new Tokenizer().setInputCol("_1").setOutputCol("words")
val remover = new StopWordsRemover().setInputCol("words").setOutputCol("filteredWords")

// A function to transform an item into a tuple of that item and a list of comments on that item
val getComments: (Item) => Seq[Comment] = (story: Item) => {
    def _getComments:  (Item) => Seq[Comment] = (item: Item) => {
        val commentIds = item.commentIds
        if(commentIds.size == 0){
            Seq(Comment(story.id.id, item.id.id, item.text))
        } else {
            val comments: Seq[Comment] = commentIds.flatMap((itemId: ItemId) => { 
                _getComments(HackerNews.getItem(itemId).get)
            })
            if("Story".equals(item.itemType.toString)){
                comments
            } else {
                Comment(story.id.id, item.id.id, item.text) +: comments
            }
            
        }   
    }
    
    _getComments(story)
}

val getItemText: (Comment) => String = (comment: Comment) => {
    Jsoup.parse(comment.text).text()
}
val getItemLinks: (Comment) => Seq[String] = (comment: Comment) => {
    val aTags: List[Element] = Jsoup.parse(comment.text).select("a").toList
    aTags.map((link: Element) => {
        link.attr("href")
    })
}

def getStoryComments(storyId: Int) = {
    val story = Seq(HackerNews.getItem(ItemId(storyId)).get)
    sc.parallelize(story).flatMap((item: Item) => {
        getComments(item)
    })
}   

def getCommentLinks(comments: RDD[Comment]) = {
    comments.flatMap((comment:Comment) => {
        getItemLinks(comment)
    })
}

def getCommentWordCounts(comments: RDD[Comment]) = {
    val textDF = comments.map((comment:Comment) => {
        getItemText(comment)
    }).toDF
    val tokenizedComments = tokenizer.transform(textDF)
    val filteredWordCountsDF = remover.transform(tokenizedComments)
    val terms = filteredWordCountsDF.flatMap((row: Row) =>{
        row.getSeq[String](2)
    })
    val wordCounts = terms.map((word: String) => {
        (word, 1)
    }).reduceByKey(_+_)
    wordCounts
}

In [ ]:
case class Story(id: Long, title: String, url: String)

In [ ]:
def getTopNStories(n: Int=10) = {
    val rdd = sc.parallelize(HackerNews.getTopStories(n))
    rdd.cache()
    val stories = rdd.map((item: Item) => {
        Story(item.id.id, item.title.get, item.url.get)
    })
    stories
}

## Adding Widgets
Now we will begin integration the widget code into our demo. This begin by importing classes and initializing the library. 

In [ ]:
import declarativewidgets._
initWidgets

The [Jupyter Declarative Widget](https://github.com/jupyter-incubator/declarativewidgets) is built off of [Polymer webcomponents](https://www.polymer-project.org/1.0/) which allows us to incorporate any of the many components found online.

In [ ]:
%%html
<link rel="import" href="urth_components/paper-card/paper-card.html"
    is="urth-core-import" package="PolymerElements/paper-card" >
<link rel="import" href="urth_components/paper-button/paper-button.html"
    is="urth-core-import" package="PolymerElements/paper-button" >

<link rel="import" href="urth_components/iron-list/iron-list.html"
    is="urth-core-import" package="PolymerElements/iron-list" >
<link rel="import" href="urth_components/paper-listbox/paper-listbox.html"
    is="urth-core-import" package="PolymerElements/paper-listbox" >
<link rel="import" href="urth_components/paper-item/paper-item.html"
    is="urth-core-import" package="PolymerElements/paper-item" >
<link rel="import" href="urth_components/juicy-html/juicy-html.html"
    is="urth-core-import" package="Juicy/juicy-html" >

These are some wrapper functions for integrating with our widgets. They mainly handle serializing the values out to the widgets.

In [ ]:
def storyComments(story: String): Seq[Map[String, String]] = {
    getStoryComments(story.toInt).collect().map((comment: Comment) => {
        Map[String, String]("text" -> comment.text)
    })
}

def storyLinks(story: String) = {
    getCommentLinks(getStoryComments(story.toInt)).collect()
}

def storyWords(story: String) = {
    getCommentWordCounts(getStoryComments(story.toInt)).sortBy((wordCount: (String, Int)) => {
        wordCount._2
    }, ascending=false).take(50)
}

def getMoreInfo(story: String) = {
    channel("default").set("status", "Loading Comments")
    val comments = storyComments(story)
    channel("default").set("status", "Loading Links")
    val links = storyLinks(story)
    channel("default").set("status", "Loading Word Counts")
    val wordCounts = storyWords(story)
    
    channel("default").set("comments", comments)
    channel("default").set("links", links)
    channel("default").set("words", wordCounts)
    channel("default").set("status", "")
    true
}

def stories() = {
    val stories = getTopNStories(10).collect().map((story: Story) => {
        Map[String, String]("id"->story.id.toString, "title"->story.title, "url"->story.url)
    })
    channel("default").set("stories", stories)
    stories
}

This is the main "UI" for the application. A list of the Top 10 stories will be displayed. The user can then select the story to inspect. 

In [ ]:
%%html
<style>
.iron-selected {
    background-color: #ddd;
}
</style>
<template is="urth-core-bind">
    <urth-core-function auto ref="stories"></urth-core-function>
    <urth-core-function auto ref="getMoreInfo" arg-story="{{selectedStoryId}}"></urth-core-function>
    <h1>Top 10 Hacker News Stories</h1>
    <p>{{status}}</p>
    <paper-listbox attr-for-selected="data-name" selected="{{selectedStoryId}}">
        <template is="dom-repeat" items="{{stories}}">
            <paper-item data-name$="{{item.id}}">
                <a href="{{item.url}}">{{item.title}}</a>
            </paper-item>
        </template>
    </paper-listbox>
</template>

In [ ]:
%%html
<h2>Comments</h2>
<template is="urth-core-bind">
    <iron-list style="height: 300px;" target="document" items="[[comments]]">
        <template>
            <paper-card>
                <div class="card-content">
                    <template is="juicy-html" content$="{{item.text}}"></template>
                </div>
            </paper-card>
        </template>
    </iron-list>
</template>

In [ ]:
%%html
<h2>Word Counts</h2>
<template is="urth-core-bind">
    <iron-list style="height: 300px;" target="document" items="[[words]]">
        <template>
            <paper-card>
                <div class="card-content">
                    <p>{{item}}</p>
                </div>
            </paper-card>
        </template>
    </iron-list>
</template>

In [ ]:
%%html
<h2>Comment Links</h2>
<template is="urth-core-bind">
    <iron-list style="height: 300px;" target="document" items="[[links]]">
        <template>
            <paper-card>
                <div class="card-content">
                    <a href="{{item}}">{{item}}</a>
                </div>
            </paper-card>
        </template>
    </iron-list>
</template>